## verificar dimansion de las imágenes

In [ ]:
import os
from PIL import Image

def check_image_sizes(folder_path):
    # Tamaño esperado de las imágenes
    expected_size = (1024, 1024)
    
    # Obtener la lista de archivos en la carpeta
    files = os.listdir(folder_path)
    
    # Filtrar solo los archivos con extensión .png
    png_files = [file for file in files if file.lower().endswith('.png')]
    
    # Inicializar una lista para almacenar los resultados
    incorrect_size_images = []
    
    # Verificar cada imagen PNG
    for file in png_files:
        image_path = os.path.join(folder_path, file)
        
        try:
            with Image.open(image_path) as img:
                if img.size != expected_size:
                    incorrect_size_images.append((file, img.size))
        except Exception as e:
            print(f"Error al abrir la imagen {file}: {e}")
    
    # Resultados
    if incorrect_size_images:
        print("Las siguientes imágenes no tienen el tamaño 1024x1024:")
        for image_info in incorrect_size_images:
            print(f"{image_info[0]} - Tamaño: {image_info[1]}")
    else:
        print("Todas las imágenes tienen el tamaño 1024x1024.")

# Ejemplo de uso
check_image_sizes("./7_dataset_normalizado_bicubica")


## Renombrar las imágenes y clasificarlas

In [ ]:
import os
import pandas as pd
import shutil

# Ruta de la carpeta donde están las imágenes PNG
input_dir = "./7_dataset_normalizado_bicubica"
output_dir = "./8_dataset_clasificado_BI-RADS"

# Lista con las rutas de los archivos CSV que contienen los nombres
archivos_csv = [
    "CC_BI-RADS_1.csv",
    "MLO_BI-RADS_1.csv",
    "CC_BI-RADS_2.csv",
    "MLO_BI-RADS_2.csv",
    "CC_BI-RADS_3.csv",
    "MLO_BI-RADS_3.csv",
    "CC_BI-RADS_4.csv",
    "MLO_BI-RADS_4.csv",
    "CC_BI-RADS_5.csv",
    "MLO_BI-RADS_5.csv"
]

# Función para renombrar las imágenes y moverlas a una subcarpeta correspondiente
def renombrar_y_mover_imagenes(carpeta_entrada, carpeta_salida, dataframe, nombre_csv):
    # Crear una subcarpeta basada en el nombre del archivo CSV
    subcarpeta = os.path.join(carpeta_salida, os.path.splitext(os.path.basename(nombre_csv))[0])
    if not os.path.exists(subcarpeta):
        os.makedirs(subcarpeta)
    
    for index, row in dataframe.iterrows():
        original_name = row[0]  # Primera columna es el nombre original
        new_name = row[1]       # Segunda columna es el nuevo nombre (incluye .png)
        
        # Agregar la extensión .png si no está presente en el nombre original
        if not original_name.endswith('.png'):
            original_name = f"{original_name}.png"
        
        # Ruta completa del archivo original
        archivo_original = os.path.join(carpeta_entrada, original_name)
        # Ruta del archivo en la subcarpeta con el nuevo nombre
        archivo_final = os.path.join(subcarpeta, new_name)
        
        # Verifica si el archivo original existe antes de copiar
        if os.path.exists(archivo_original):
            try:
                # Copiar el archivo y renombrarlo en la carpeta de salida
                shutil.copy(archivo_original, archivo_final)
                print(f"Archivo {original_name} copiado y renombrado como {new_name} en {subcarpeta}")
            except Exception as e:
                print(f"Error al copiar {original_name}: {e}")
        else:
            print(f"El archivo {original_name} no existe en la carpeta de entrada.")

# Procesa todos los archivos CSV en la lista
for archivo_csv in archivos_csv:
    # Lee cada archivo CSV sin encabezado
    df = pd.read_csv(archivo_csv, header=None)
    # Renombra y mueve las imágenes basadas en el CSV actual
    renombrar_y_mover_imagenes(input_dir, output_dir, df, archivo_csv)

print("Todos los archivos han sido renombrados y movidos a sus respectivas subcarpetas.")

## distribución train/test

### Explicación del flujo:
* **Lectura de imágenes:** Se obtienen las imágenes de las carpetas CC y MLO con nombres idénticos.
* **Validación de coincidencia:** Se verifica que las listas de imágenes en CC y MLO coincidan. Si no coinciden, se emite un error.
* **Barajado único:** Se baraja la lista de imágenes utilizando random.shuffle() con una semilla (random.seed(42)) para asegurar que el barajado sea siempre reproducible.
* **División sincronizada:** La lista barajada se divide en conjuntos de entrenamiento (train) y prueba (test), y las imágenes correspondientes en CC y MLO se mueven de manera sincronizada a sus respectivas carpetas de salida.

In [ ]:
import os
import random
import shutil

# Función para copiar las imágenes a las carpetas correspondientes
def copiar_imagenes(image_list, source_dir_cc, source_dir_mlo, target_dir_cc, target_dir_mlo):
    for image in image_list:
        # Copiar imágenes de la carpeta CC
        source_path_cc = os.path.join(source_dir_cc, image)
        target_path_cc = os.path.join(target_dir_cc, image)
        shutil.copy(source_path_cc, target_path_cc)
        print(f'Copiando {image} a {target_dir_cc}')

        # Copiar imágenes de la carpeta MLO
        source_path_mlo = os.path.join(source_dir_mlo, image)
        target_path_mlo = os.path.join(target_dir_mlo, image)
        shutil.copy(source_path_mlo, target_path_mlo)
        print(f'Copiando {image} a {target_dir_mlo}')

# Función para procesar CC y MLO de manera sincronizada
def procesar_birads_sincronizado(birads_label, test_ratio=0.2, train_ratio=0.8):
    # Carpetas de entrada
    input_dir_cc = f'./8_dataset_clasificado_BI-RADS/CC_{birads_label}'
    input_dir_mlo = f'./8_dataset_clasificado_BI-RADS/MLO_{birads_label}'
    
    # Carpetas de salida dentro de 9_dataset
    output_dir = './9_prueba_dataset_separado_traintest'  # Carpeta de salida principal

    # Definir las carpetas de salida para train y test
    train_dir_cc = os.path.join(output_dir, f'CC_{birads_label}_train')
    test_dir_cc = os.path.join(output_dir, f'CC_{birads_label}_test')
    train_dir_mlo = os.path.join(output_dir, f'MLO_{birads_label}_train')
    test_dir_mlo = os.path.join(output_dir, f'MLO_{birads_label}_test')

    # Crear las carpetas de destino si no existen
    os.makedirs(train_dir_cc, exist_ok=True)
    os.makedirs(test_dir_cc, exist_ok=True)
    os.makedirs(train_dir_mlo, exist_ok=True)
    os.makedirs(test_dir_mlo, exist_ok=True)

    # Obtener una lista de todas las imágenes en la carpeta de entrada CC (las imágenes deben ser las mismas en CC y MLO)
    imagenes_cc = [f for f in os.listdir(input_dir_cc) if f.endswith('.png')]
    imagenes_mlo = [f for f in os.listdir(input_dir_mlo) if f.endswith('.png')]

    # Asegurarse de que las listas de imágenes en CC y MLO coincidan
    if set(imagenes_cc) != set(imagenes_mlo):
        print(f"Las imágenes en CC_{birads_label} y MLO_{birads_label} no coinciden.")
        return

    # Usar una lista única, ya que los nombres son iguales en CC y MLO
    imagenes = imagenes_cc

    # Barajar la lista de imágenes para asegurar la aleatoriedad
    random.seed(42)  # Usar una semilla para que el barajado sea reproducible
    random.shuffle(imagenes)

    # Calcular la cantidad de imágenes para cada conjunto
    test_size = int(len(imagenes) * test_ratio)
    train_size = len(imagenes) - test_size

    # Dividir las imágenes entre train y test
    test_images = imagenes[:test_size]
    train_images = imagenes[test_size:]

    # Copiar imágenes a las carpetas de test y train para CC y MLO de manera sincronizada
    copiar_imagenes(test_images, input_dir_cc, input_dir_mlo, test_dir_cc, test_dir_mlo)
    copiar_imagenes(train_images, input_dir_cc, input_dir_mlo, train_dir_cc, train_dir_mlo)

    print(f"Distribución de imágenes completada para {birads_label} (CC y MLO).")

# Lista de etiquetas BI-RADS a procesar (etiquetas compartidas para CC y MLO)
birads_folders = [
    "BI-RADS_1", "BI-RADS_2",
    "BI-RADS_3", "BI-RADS_4",
    "BI-RADS_5"
]

# Procesar cada conjunto de datos BI-RADS
for birads_folder in birads_folders:
    procesar_birads_sincronizado(birads_folder)

## organizar las carpetas en subcarpetas CC/MLO y train/test

In [ ]:
import os
import shutil

# Función para mover o copiar las imágenes a las subcarpetas correspondientes en CC y MLO y mantener la clasificación BI-RADS
def mover_a_subcarpetas(source_dir, target_dir_cc, target_dir_mlo):
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.png'):
                # Obtener el nombre de la carpeta base para verificar si es "train" o "test"
                folder_name = os.path.basename(root)
                
                # Verificamos la carpeta de origen (root) para saber si es CC o MLO y si es train o test
                if "CC" in root:
                    # Identificamos el BI-RADS
                    for birads in ['BI-RADS_1', 'BI-RADS_2', 'BI-RADS_3', 'BI-RADS_4', 'BI-RADS_5']:
                        if birads in root:
                            if "train" in folder_name:  # Cambiado a verificar la carpeta base
                                dest_dir = os.path.join(target_dir_cc, 'train', birads)
                            elif "test" in folder_name:  # Cambiado a verificar la carpeta base
                                dest_dir = os.path.join(target_dir_cc, 'test', birads)
                            else:
                                print(f'Carpeta no identificada correctamente: {folder_name}')
                                continue
                            # Crear la carpeta si no existe
                            os.makedirs(dest_dir, exist_ok=True)
                            shutil.copy(os.path.join(root, file), os.path.join(dest_dir, file))
                            print(f'Copiando {file} desde {root} a {dest_dir}')
                elif "MLO" in root:
                    # Identificamos el BI-RADS
                    for birads in ['BI-RADS_1', 'BI-RADS_2', 'BI-RADS_3', 'BI-RADS_4', 'BI-RADS_5']:
                        if birads in root:
                            if "train" in folder_name:  # Cambiado a verificar la carpeta base
                                dest_dir = os.path.join(target_dir_mlo, 'train', birads)
                            elif "test" in folder_name:  # Cambiado a verificar la carpeta base
                                dest_dir = os.path.join(target_dir_mlo, 'test', birads)
                            else:
                                print(f'Carpeta no identificada correctamente: {folder_name}')
                                continue
                            # Crear la carpeta si no existe
                            os.makedirs(dest_dir, exist_ok=True)
                            shutil.copy(os.path.join(root, file), os.path.join(dest_dir, file))
                            print(f'Copiando {file} desde {root} a {dest_dir}')

# Función principal para crear la estructura CC y MLO, y clasificar imágenes
def clasificar_en_cc_mlo(source_dir, output_dir):
    # Carpetas de salida CC y MLO dentro de 10_dataset_classified
    cc_dir = os.path.join(output_dir, 'CC')
    mlo_dir = os.path.join(output_dir, 'MLO')

    # Crear carpetas CC y MLO si no existen
    os.makedirs(cc_dir, exist_ok=True)
    os.makedirs(mlo_dir, exist_ok=True)

    # Llamar a la función para mover las imágenes a las carpetas correspondientes
    mover_a_subcarpetas(source_dir, cc_dir, mlo_dir)

    print("Clasificación en subcarpetas CC y MLO completada.")

# Directorio de entrada (carpeta 9_dataset)
source_dir = './9_prueba_dataset_separado_traintest'

# Directorio de salida (donde se organizarán las imágenes en CC y MLO)
output_dir = './Dataset_desbalanceado'

# Clasificar las imágenes en las subcarpetas CC y MLO
clasificar_en_cc_mlo(source_dir, output_dir)

# **Balanceo**

* Inversión horizontal en todas las etiquetas, agregando un _a en su nombre

In [ ]:
from PIL import Image
import os

def copy_and_flip_image(image_path, output_path):
    # Abrir la imagen
    img = Image.open(image_path)
    
    # Guardar la imagen original en la carpeta de salida
    img.save(output_path)

    # Invertir horizontalmente
    flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
    
    # Crear el nombre del archivo de salida con el sufijo '_a' antes de la extensión
    base_name, ext = os.path.splitext(output_path)
    new_image_path = base_name + '_a' + ext

    # Guardar la imagen invertida con el nuevo nombre
    flipped_img.save(new_image_path)

def process_images_in_folders(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Recorrer todas las subcarpetas y archivos en la carpeta de entrada
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith(('.png')): 
                image_path = os.path.join(root, file)
                
                # Crear la ruta de salida para mantener la estructura de carpetas
                relative_path = os.path.relpath(root, input_folder)
                output_dir = os.path.join(output_folder, relative_path)
                
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                
                # Crear la ruta de salida para la imagen original
                output_path = os.path.join(output_dir, file)

                # Guardar la imagen original e invertida en la nueva carpeta
                copy_and_flip_image(image_path, output_path)
                print(f"Procesada la imagen: {output_path}")

# Definir las rutas de entrada y salida
input_folder = 'Dataset_desbalanceado'  # Cambia esto por la ruta correcta
output_folder = 'Dataset_desbalanceado_a'  # Carpeta de salida para las imágenes originales e invertidas

# Procesar todas las imágenes
process_images_in_folders(input_folder, output_folder)

* Zoom 20%, agregando un _b en su nombre

In [ ]:
'''# en una sola imagen
from PIL import Image
import os

def apply_zoom(image_path, zoom_factor=1.2):
    # Abrir la imagen
    img = Image.open(image_path)
    
    # Obtener el tamaño original
    width, height = img.size
    
    # Calcular el nuevo tamaño (aplicando el zoom)
    new_width = int(width * zoom_factor)
    new_height = int(height * zoom_factor)
    
    # Redimensionar la imagen (aplicando el zoom)
    zoomed_img = img.resize((new_width, new_height), Image.LANCZOS)
    
    # Recortar la imagen para mantener el tamaño original (centrar la imagen)
    left = (new_width - width) // 2
    top = (new_height - height) // 2
    right = left + width
    bottom = top + height
    
    cropped_img = zoomed_img.crop((left, top, right, bottom))
    
    # Extraer el nombre base y la extensión del archivo
    base_name, ext = os.path.splitext(image_path)
    
    # Crear el nuevo nombre de archivo agregando el sufijo '_c'
    new_image_path = base_name + '_b' + ext
    
    # Guardar la imagen modificada
    cropped_img.save(new_image_path)
    
    print(f"Imagen con zoom guardada como: {new_image_path}")

# Ejemplo de uso
image_path = '0f1a7b0d4efecb388a4b98ae5e2d5c29_R.png'  # Reemplaza esto con la ruta de la imagen que deseas procesar
apply_zoom(image_path, zoom_factor=1.2)
'''

In [ ]:
from PIL import Image
import os
import shutil

def apply_zoom(image_path, output_path, zoom_factor=1.2):
    # Abrir la imagen
    img = Image.open(image_path)
    
    # Obtener el tamaño original
    width, height = img.size
    
    # Calcular el nuevo tamaño (aplicando el zoom)
    new_width = int(width * zoom_factor)
    new_height = int(height * zoom_factor)
    
    # Redimensionar la imagen (aplicando el zoom)
    zoomed_img = img.resize((new_width, new_height), Image.LANCZOS)
    
    # Recortar la imagen para mantener el tamaño original (centrar la imagen)
    left = (new_width - width) // 2
    top = (new_height - height) // 2
    right = left + width
    bottom = top + height
    
    cropped_img = zoomed_img.crop((left, top, right, bottom))
    
    # Guardar la imagen con zoom
    cropped_img.save(output_path)

def process_images_in_folders(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Recorrer todas las subcarpetas y archivos en la carpeta de entrada
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith(('.png')):  # Ajusta los formatos de imagen si es necesario
                image_path = os.path.join(root, file)
                
                # Crear la ruta de salida para mantener la estructura de carpetas
                relative_path = os.path.relpath(root, input_folder)
                output_dir = os.path.join(output_folder, relative_path)
                
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                
                # Crear la ruta de salida para la imagen original
                output_path = os.path.join(output_dir, file)
                
                # Copiar la imagen original
                shutil.copy(image_path, output_path)
                print(f"Imagen original copiada: {output_path}")
                
                # Verificar si estamos en 'BI-RADS_1'
                if 'BI-RADS_1' in root:
                    # Si es 'BI-RADS_1' no aplicar zoom, solo se copiar la original
                    continue
                else:
                    # Si no es 'BI-RADS_1' o 'BI-RADS_2', aplicar zoom y guardar con sufijo '_b'
                    base_name, ext = os.path.splitext(file)
                    new_file_name = base_name + '_b' + ext
                    new_output_path = os.path.join(output_dir, new_file_name)
                    
                    # Aplicar el zoom y guardar la imagen con el sufijo '_b'
                    apply_zoom(image_path, new_output_path)
                    print(f"Imagen con zoom guardada como: {new_output_path}")

# Definir las rutas de entrada y salida
input_folder = 'Dataset_desbalanceado_a'  # Cambia esto por la ruta correcta
output_folder = 'Dataset_desbalanceado_a_b'  # Carpeta de salida para las imágenes originales y procesadas

# Procesar todas las imágenes, excepto en 'BI-RADS_1'
process_images_in_folders(input_folder, output_folder)

* Aumentar nitidez, agregando un _c en su nombre

In [ ]:
'''# en una sola imagen
from PIL import Image, ImageEnhance
import os

def enhance_sharpness(image_path, sharpness_factor=5):
    # Abrir la imagen
    img = Image.open(image_path)
    
    # Aumentar la nitidez utilizando ImageEnhance
    enhancer = ImageEnhance.Sharpness(img)
    
    # Aplicar el factor de nitidez (puedes ajustar el valor según sea necesario)
    sharp_img = enhancer.enhance(sharpness_factor)
    
    # Extraer el nombre base y la extensión del archivo
    base_name, ext = os.path.splitext(image_path)
    
    # Crear el nuevo nombre de archivo agregando el sufijo '_c'
    new_image_path = base_name + '_c' + ext
    
    # Guardar la imagen con mayor nitidez
    sharp_img.save(new_image_path)
    
    print(f"Imagen con nitidez aumentada guardada como: {new_image_path}")

# Ejemplo de uso
image_path = '0f1a7b0d4efecb388a4b98ae5e2d5c29_R.png'  # Reemplaza con la ruta de la imagen que deseas procesar
enhance_sharpness(image_path, sharpness_factor=5)
'''

In [ ]:
from PIL import Image, ImageEnhance
import os
import shutil

def enhance_sharpness(image_path, output_path, sharpness_factor=5.0):
    # Abrir la imagen
    img = Image.open(image_path)
    
    # Aumentar la nitidez utilizando ImageEnhance
    enhancer = ImageEnhance.Sharpness(img)
    
    # Aplicar el factor de nitidez
    sharp_img = enhancer.enhance(sharpness_factor)
    
    # Guardar la imagen con mayor nitidez
    sharp_img.save(output_path)

def process_images_in_folders(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Recorrer todas las subcarpetas y archivos en la carpeta de entrada
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith(('.png')):  # Ajusta los formatos de imagen si es necesario
                image_path = os.path.join(root, file)
                
                # Crear la ruta de salida para mantener la estructura de carpetas
                relative_path = os.path.relpath(root, input_folder)
                output_dir = os.path.join(output_folder, relative_path)
                
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                
                # Crear la ruta de salida para la imagen original
                output_path = os.path.join(output_dir, file)
                
                # Copiar la imagen original
                shutil.copy(image_path, output_path)
                print(f"Imagen original copiada: {output_path}")
                
                # Verificar si estamos en 'BI-RADS_1' o 'BI-RADS_2'
                if 'BI-RADS_1' in root or 'BI-RADS_2' in root:
                    # Si es 'BI-RADS_1' o 'BI-RADS_2', no aplicar nitidez, solo se copiar la original
                    continue
                else:
                    # Si no está en 'BI-RADS_1' o 'BI-RADS_2', aplicar nitidez y guardar con sufijo '_c'
                    base_name, ext = os.path.splitext(file)
                    new_file_name = base_name + '_c' + ext
                    new_output_path = os.path.join(output_dir, new_file_name)
                    
                    # Aplicar la nitidez y guardar la imagen con el sufijo '_c'
                    enhance_sharpness(image_path, new_output_path, sharpness_factor=5.0)
                    print(f"Imagen con nitidez aumentada guardada como: {new_output_path}")

# Definir las rutas de entrada y salida
input_folder = 'Dataset_desbalanceado_a_b'  # Cambia esto por la ruta correcta
output_folder = 'Dataset_desbalanceado_a_b_c'  # Carpeta de salida para las imágenes originales y procesadas

# Procesar todas las imágenes, excepto en 'BI-RADS_1' y 'BI-RADS_2'
process_images_in_folders(input_folder, output_folder)

* Aumentar contraste, agregando un _d en su nombre

In [ ]:
import os
import cv2
import shutil

# Función para aplicar CLAHE y mejorar el contraste de manera sutil
def enhance_contrast(image_path, output_path, clip_limit=2.0, tile_grid_size=(8, 8)):
    # Cargar la imagen en escala de grises
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Paso 1: Normalización de intensidad para imágenes muy brillantes
    normalized_img = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    
    # Paso 2: Aplicar CLAHE para mejorar el contraste de forma sutil
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    enhanced_img = clahe.apply(normalized_img)
    
    # Guardar la imagen procesada
    cv2.imwrite(output_path, enhanced_img)
    
def process_images_in_folders(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Recorrer todas las subcarpetas y archivos en la carpeta de entrada
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.png'):
                image_path = os.path.join(root, file)
                
                # Crear la ruta de salida para mantener la estructura de carpetas
                relative_path = os.path.relpath(root, input_folder)
                output_dir = os.path.join(output_folder, relative_path)
                
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                
                # Crear la ruta de salida para la imagen original
                output_path = os.path.join(output_dir, file)
                
                # Copiar la imagen original
                shutil.copy(image_path, output_path)
                print(f"Imagen original copiada: {output_path}")
                
                # Verificar si estamos en 'BI-RADS_1' o 'BI-RADS_2'
                if 'BI-RADS_1' in root or 'BI-RADS_2' in root:
                    # Si es 'BI-RADS_1' o 'BI-RADS_2', no aplicar contraste, solo copiar la original
                    continue
                else:
                    # Si no está en 'BI-RADS_1' o 'BI-RADS_2', aplicar contraste y guardar con sufijo '_d'
                    base_name, ext = os.path.splitext(file)
                    new_file_name = base_name + '_d' + ext
                    new_output_path = os.path.join(output_dir, new_file_name)
                    
                    # Aplicar el aumento de contraste y guardar la imagen con el sufijo '_d'
                    enhance_contrast(image_path, new_output_path)
                    print(f"Imagen con contraste aumentado guardada como: {new_output_path}")

# Definir las rutas de entrada y salida
input_folder = 'Dataset_desbalanceado_a_b_c'  # Cambia esto por la ruta correcta
output_folder = 'Dataset_desbalanceado_a_b_c_d'  # Carpeta de salida para las imágenes originales y procesadas

# Procesar todas las imágenes, excepto en 'BI-RADS_1' y 'BI-RADS_2'
process_images_in_folders(input_folder, output_folder)

* Aumentar brillo 1.07, agregando un _e en su nombre

In [ ]:
'''# en una sola imagen
from PIL import Image, ImageEnhance
import os

def adjust_brightness(image_path, brightness_factor=1.07):
    # Abrir la imagen
    img = Image.open(image_path)
    
    # Disminuir el brillo utilizando ImageEnhance
    enhancer = ImageEnhance.Brightness(img)
    
    # Aplicar el factor de brillo
    bright_img = enhancer.enhance(brightness_factor)
    
    # Extraer el nombre base y la extensión del archivo
    base_name, ext = os.path.splitext(image_path)
    
    # Crear el nuevo nombre de archivo agregando el sufijo '_e'
    new_image_path = base_name + '_e' + ext
    
    # Guardar la imagen con brillo disminuido
    bright_img.save(new_image_path)
    
    print(f"Imagen con brillo aumentado guardada como: {new_image_path}")

# Ejemplo de uso
image_path = '2c536e8b72454ad979b24648db6d7af0_L_a_b_d.png'  # Reemplaza con la ruta de la imagen que deseas procesar
adjust_brightness(image_path, brightness_factor=1.07)'''

In [ ]:
from PIL import Image, ImageEnhance
import os
import shutil

def adjust_brightness(image_path, output_path, brightness_factor=1.07):
    # Abrir la imagen
    img = Image.open(image_path)
    
    # Aumentar el brillo utilizando ImageEnhance
    enhancer = ImageEnhance.Brightness(img)
    
    # Aplicar el factor de brillo
    bright_img = enhancer.enhance(brightness_factor)
    
    # Guardar la imagen con brillo aumentado
    bright_img.save(output_path)

def process_images_in_folders(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Recorrer todas las subcarpetas y archivos en la carpeta de entrada
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith(('.png')):  # Ajusta los formatos de imagen si es necesario
                image_path = os.path.join(root, file)
                
                # Crear la ruta de salida para mantener la estructura de carpetas
                relative_path = os.path.relpath(root, input_folder)
                output_dir = os.path.join(output_folder, relative_path)
                
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                
                # Crear la ruta de salida para la imagen original
                output_path = os.path.join(output_dir, file)
                
                # Copiar la imagen original
                shutil.copy(image_path, output_path)
                print(f"Imagen original copiada: {output_path}")
                
                # Verificar si estamos en 'BI-RADS_1' o 'BI-RADS_2' o 'BI-RADS_3'
                if 'BI-RADS_1' in root or 'BI-RADS_2' in root or 'BI-RADS_3' in root:
                    # Si es 'BI-RADS_1' o 'BI-RADS_2' o 'BI-RADS_3', no aplicar contraste, solo se copiar la original
                    continue
                else:
                    # Si no está en 'BI-RADS_1' o 'BI-RADS_2' o 'BI-RADS_3', aplicar contraste y guardar con sufijo '_e'
                    base_name, ext = os.path.splitext(file)
                    new_file_name = base_name + '_e' + ext
                    new_output_path = os.path.join(output_dir, new_file_name)
                    
                    # Aplicar el aumento de contraste y guardar la imagen con el sufijo '_e'
                    adjust_brightness(image_path, new_output_path, brightness_factor=1.07)
                    print(f"Imagen con contraste aumentado guardada como: {new_output_path}")

# Definir las rutas de entrada y salida
input_folder = 'Dataset_desbalanceado_a_b_c_d'  # Cambia esto por la ruta correcta
output_folder = 'Dataset_desbalanceado_a_b_c_d_e'  # Carpeta de salida para las imágenes originales y procesadas

# Procesar todas las imágenes, excepto en 'BI-RADS_1' y 'BI-RADS_2'
process_images_in_folders(input_folder, output_folder) 

* Disminuir Brillo 0.8, agregando un _f en su nombre

In [ ]:
'''# en una sola imagen
from PIL import Image, ImageEnhance
import os

def decrease_brightness(image_path, brightness_factor=0.5):
    # Abrir la imagen
    img = Image.open(image_path)
    
    # Disminuir el brillo utilizando ImageEnhance
    enhancer = ImageEnhance.Brightness(img)
    
    # Aplicar el factor de brillo (menor a 1.0 para disminuir el brillo)
    bright_img = enhancer.enhance(brightness_factor)
    
    # Extraer el nombre base y la extensión del archivo
    base_name, ext = os.path.splitext(image_path)
    
    # Crear el nuevo nombre de archivo agregando el sufijo '_e'
    new_image_path = base_name + '_f' + ext
    
    # Guardar la imagen con brillo disminuido
    bright_img.save(new_image_path)
    
    print(f"Imagen con brillo disminuido guardada como: {new_image_path}")

# Ejemplo de uso
image_path = '0f1a7b0d4efecb388a4b98ae5e2d5c29_R.png'  # Reemplaza con la ruta de la imagen que deseas procesar
decrease_brightness(image_path, brightness_factor=0.5)
'''

In [ ]:
from PIL import Image, ImageEnhance
import os
import shutil

def decrease_brightness(image_path, output_path, brightness_factor=0.8):
    # Abrir la imagen
    img = Image.open(image_path)
    
    # Disminuir el brillo utilizando ImageEnhance
    enhancer = ImageEnhance.Brightness(img)
    
    # Aplicar el factor de brillo
    bright_img = enhancer.enhance(brightness_factor)
    
    # Guardar la imagen con brillo disminuido
    bright_img.save(output_path)

def process_images_in_folders(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Recorrer todas las subcarpetas y archivos en la carpeta de entrada
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith(('.png')):  # Ajusta los formatos de imagen si es necesario
                image_path = os.path.join(root, file)
                
                # Crear la ruta de salida para mantener la estructura de carpetas
                relative_path = os.path.relpath(root, input_folder)
                output_dir = os.path.join(output_folder, relative_path)
                
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                
                # Crear la ruta de salida para la imagen original
                output_path = os.path.join(output_dir, file)
                
                # Copiar la imagen original
                shutil.copy(image_path, output_path)
                print(f"Imagen original copiada: {output_path}")
                
                # Verificar si estamos en 'BI-RADS_5'
                if 'BI-RADS_5' in root:
                    # Aplicar la disminución de brillo y guardar la imagen con sufijo '_e'
                    base_name, ext = os.path.splitext(file)
                    new_file_name = base_name + '_f' + ext
                    new_output_path = os.path.join(output_dir, new_file_name)
                    
                    decrease_brightness(image_path, new_output_path, brightness_factor=0.8)
                    print(f"Imagen con brillo disminuido guardada como: {new_output_path}")
                   
# Definir las rutas de entrada y salida
input_folder = 'Dataset_desbalanceado_a_b_c_d_e'  # Cambia esto por la ruta correcta
output_folder = 'Dataset_desbalanceado_a_b_c_d_e_f'  # Carpeta de salida para las imágenes originales y procesadas

# Procesar todas las imágenes
process_images_in_folders(input_folder, output_folder)